In [11]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Bidirectional
import re

file1 = open("C://Eday//docs//Datascience//JHU//en_US.blogs.txt",mode='r',encoding="utf-8")
file2 = open("C://Eday//docs//Datascience//JHU//en_US.news.txt",mode='r',encoding="utf-8")
file3 = open("C://Eday//docs//Datascience//JHU//en_US.twitter.txt",mode='r',encoding="utf-8")

US_blogs = file1.readlines()
US_news = file2.readlines()
US_twitter = file3.readlines()

text = US_blogs[1:1500] + US_news[1:1500] + US_twitter[1:1500]


In [12]:
len(text)

4497

In [54]:
text

['Chad has been awesome with the kids and holding down the fort while I work later than usual! The kids have been busy together playing Skylander on the XBox together, after Kyan cashed in his $$$ from his piggy bank. He wanted that game so bad and used his gift card from his birthday he has been saving and the money to get it (he never taps into that thing either, that is how we know he wanted it so bad). We made him count all of his money to make sure that he had enough! It was very cute to watch his reaction when he realized he did! He also does a very good job of letting Lola feel like she is playing too, by letting her switch out the characters! She loves it almost as much as him.\n',
 'so anyways, i am going to share some home decor inspiration that i have been storing in my folder on the puter. i have all these amazing images stored away ready to come to life when we get our home.\n',
 "With graduation season right around the corner, Nancy has whipped up a fun set to help you ou

In [13]:
import re
from bs4 import BeautifulSoup
import unicodedata
import contractions
import spacy
import nltk

nlp = spacy.load('en')
ps = nltk.porter.PorterStemmer()


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)


def spacy_lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text


def simple_stemming(text, stemmer=ps):
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text


def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


def remove_stopwords(text, is_lower_case=False, stopwords=None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopwords]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
    
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [14]:
import tqdm

def text_pre_processor(text, html_strip=True, accented_char_removal=True, contraction_expansion=True,
                       text_lower_case=True, text_stemming=False, text_lemmatization=True, 
                       special_char_removal=True, remove_digits=True, stopword_removal=True, 
                       stopword_list=None):
    
    # strip HTML
    if html_strip:
        text = strip_html_tags(text)
    
    # remove extra newlines (often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    if accented_char_removal:
        text = remove_accented_chars(text)
    
    # expand contractions    
    if contraction_expansion:
        text = expand_contractions(text)
        
    
    # lemmatize text
    if text_lemmatization:
        text = spacy_lemmatize_text(text) 
        
    # remove special characters and\or digits    
    if special_char_removal:
        # insert spaces between special characters to isolate them    
        special_char_pattern = re.compile(r'([{.(-)!}])')
        text = special_char_pattern.sub(" \\1 ", text)
        text = remove_special_characters(text, remove_digits=remove_digits)  
        
    # stem text
    if text_stemming and not text_lemmatization:
        text = simple_stemming(text)
        
    # lowercase the text    
    if text_lower_case:
        text = text.lower()
        
        
    # remove stopwords
    if stopword_removal:
        text = remove_stopwords(text, is_lower_case=text_lower_case, 
                                stopwords=stopword_list)
        
    # remove extra whitespace
    text = re.sub(' +', ' ', text)
    text = text.strip()
    
    return text

def corpus_pre_processor(corpus):
    norm_corpus = []
    for doc in tqdm.tqdm(corpus):
        norm_corpus.append(text_pre_processor(doc))
    return norm_corpus


In [15]:
data = [ text_pre_processor(line) for line in text ]

In [16]:
len(data)

4497

In [17]:
data[1:2]

['chad awesome kid hold fort work later usual kid busy together play skylander xbox together kyan cash piggy bank want game bad use gift card birthday save money get never tap thing either know want bad make count money make sure enough cute watch reaction realize also good job let lola feel like play let switch character love almost much']

In [18]:
#wordlist = ["hi what's up home diddle mc doo", "Oh wise master kakarot","Hi Hello"]
data_limit = list(map(lambda x: ' '.join(x.split(' ')[:100]), data))

In [19]:
len(data_limit)

4497

In [20]:
import gc
gc.collect()

383

In [103]:
data_limit[50:55]

['whether look invest residential commercial industrial property urban redevelopment authoritys ura area planning guideline long term impact propertys value',
 'thus begin frantic sprint corridor pull reluctant suitcase bob anything path',
 'star make magazine sexiest women alive list include model turn actress rosie huntington whiteley jazz artist esperanza spalding friends benefit star mila kunis help actress emma stone singer beyonce',
 'ah parga many time sit paxos look across mainland night see twinkling light far distance many time wonder like well know unfortunately place twinkle light parga little community get wrong parga itinerary anyway conclude worth detour two resort bay separate ancient turkish fort steep hill cluster souvenir shop taverna restaurant marina unwelcoming moor alongside rusting capsized ferry trudge buy provision take us along neglect rubbish strew track thomas would leave boat fear rob hey pull gangplank crack open beer experiment cooking galley',
 'blakema

In [59]:
len(data)

19

In [21]:
data[1:4]

['chad awesome kid hold fort work later usual kid busy together play skylander xbox together kyan cash piggy bank want game bad use gift card birthday save money get never tap thing either know want bad make count money make sure enough cute watch reaction realize also good job let lola feel like play let switch character love almost much',
 'anyways go share home decor inspiration store folder puter amazing image store away ready come life get home',
 'graduation season right around corner nancy whip fun set help graduation card gift occasion bring change one life stamp image memento tuxedo black cut circle nestabilities emboss kraft red cardstock te new stars impressions plate double side give fantastic pattern see use impressions plates tutorial taylor create one pass die cut machine use embossing pad kit need super easy']

In [105]:
# data_red = [ data.split(' ') for line in text ]
#res = [][]
#res = [ ''.joword for line in data for word in line.split(' ') ]
#sub.split()[N - 1] 
# [j for i in range(100) if i > 10 for j in range(i) if j < 20]
#if len(sub.split()) > 1
#print(len(res),res)

In [22]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [23]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
#print(tokenizer.index_word)
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# create line-based sequences
sequences = list()
#for line in data.split('\n'):
for line in text:
	encoded = tokenizer.texts_to_sequences([line])[0]
	#for i in range(1, len(encoded)):
	for i in range(1, 100):
		sequence = encoded[:i+1]
		sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
#print(sequences)
# pad input sequences
max_length = max([len(seq) for seq in sequences])
#sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
sequences = pad_sequences(sequences, maxlen=100, padding='pre')
print('Max Sequence Length: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

Vocabulary Size: 13657
Total Sequences: 445203
Max Sequence Length: 100


In [24]:
max_length

100

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 20, input_length=max_length-1))
#model.add(LSTM(50))
model.add(Bidirectional(LSTM(25, return_sequences = True)))
model.add(Bidirectional(LSTM(25, return_sequences = False)))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=50, verbose=2)
# evaluate model
#print(generate_seq(model, tokenizer, max_length-1, 'Jack', 4))
#print(generate_seq(model, tokenizer, max_length-1, 'Jill', 4))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 99, 20)            273140    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 99, 50)            9200      
_________________________________________________________________
bidirectional_6 (Bidirection (None, 50)                15200     
_________________________________________________________________
dense_3 (Dense)              (None, 13657)             696507    
Total params: 994,047
Trainable params: 994,047
Non-trainable params: 0
_________________________________________________________________
None


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
 - 3357s - loss: 6.1366 - accuracy: 0.0572
Epoch 2/50


Jack and Jill went up the hill to


In [39]:
print(generate_seq(model, tokenizer, max_length-1, 'Sullivan called on CPS', 5))

Sullivan called on CPS correct improve employee accountability example


In [28]:
max_length

2

In [30]:
tokenizer.word_index

{'and': 1,
 'jack': 2,
 'jill': 3,
 'went': 4,
 'up': 5,
 'the': 6,
 'hill': 7,
 'to': 8,
 'fetch': 9,
 'a': 10,
 'pail': 11,
 'of': 12,
 'water': 13,
 'fell': 14,
 'down': 15,
 'broke': 16,
 'his': 17,
 'crown': 18,
 'came': 19,
 'tumbling': 20,
 'after': 21}

In [16]:
I was just trying to hit it hard someplace

In [31]:
joblib.dump(model,'word_pred_sim.pkl')

['word_pred_sim.pkl']

In [32]:
#joblib.dump(tokenizer,'vectorizer.pkl')

['vectorizer.pkl']

In [33]:
# saving
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer1 = pickle.load(handle)

In [34]:
type(tokenizer)

keras_preprocessing.text.Tokenizer

In [35]:
tokenizer.index_word

{1: 'and',
 2: 'jack',
 3: 'jill',
 4: 'went',
 5: 'up',
 6: 'the',
 7: 'hill',
 8: 'to',
 9: 'fetch',
 10: 'a',
 11: 'pail',
 12: 'of',
 13: 'water',
 14: 'fell',
 15: 'down',
 16: 'broke',
 17: 'his',
 18: 'crown',
 19: 'came',
 20: 'tumbling',
 21: 'after'}